In [1]:
import os
import re
import json
import gzip
import typing
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('../')

In [3]:
from shared.utils.loaders.df import json_gzip_into_df, df_into_csv_gzip
from shared.utils.pre_processing import CommonPreProcessing, PricePreProcessing

## Loading Data

In [4]:
df = json_gzip_into_df('../shared/data/amz_products_small.jsonl.gz')

In [5]:
df.head(3)

,also_buy,also_view,asin,brand,category,description,feature,image,price,title,main_cat
0,[],[],B00ADZ3WUM,NSI,"[Automotive, Exterior Accessories, Bumper Stic...",[Bumpersticker: A day without sunshine is like...,[Official Licensed Die-Cut Sticker Designed by...,[],$4.68,"NSI - A Day Without Sunshine is Like, Well, Ni...",Automotive
1,[],[],B005VII5IU,General Motors,"[Automotive, Replacement Parts, Body & Trim, B...",[This is the official Genuine General Motors P...,[This is the official Genuine General Motors P...,[],$213.16,Genuine GM Parts 10341533 Rear Bumper Valance ...,Automotive
2,[],[],B001QTEKVO,JLM,"[Automotive, Lights & Lighting Accessories, Li...",[HID Xenon lights are designed to be at least ...,"[Will run for approx 2500 hours, Produces 2 to...",[https://images-na.ssl-images-amazon.com/image...,,JLM HID Conversion Kit H13 (9008) Dual Tube B...,Automotive


## Pre-Processing Data

In [6]:
df['title'].nunique()

1214935

In [7]:
df.isnull().sum(axis=0)

also_buy       0
also_view      0
asin           0
brand          0
category       0
description    0
feature        0
image          0
price          0
title          0
main_cat       0
dtype: int64

In [13]:
# Would remove the 'category' column
df.drop(['category'], axis=1, inplace=True)

#### General Pre-Processing

Some of the data types are incorrect, and also we will handle the nulls for the columns of:
- brand
- also_buy
- also_view
- feature
- image
- price

In [14]:
common_pre_processing = CommonPreProcessing(df)

First, we will handle the missing values by putting a None in the dataframe.

In [15]:
common_pre_processing.handle_nan_columns().head()

,also_buy,also_view,asin,brand,description,feature,image,price,title,main_cat
0,None,None,B00ADZ3WUM,NSI,(Bumpersticker: A day without sunshine is like...,(Official Licensed Die-Cut Sticker Designed by...,None,$4.68,"NSI - A Day Without Sunshine is Like, Well, Ni...",Automotive
1,None,None,B005VII5IU,General Motors,(This is the official Genuine General Motors P...,(This is the official Genuine General Motors P...,None,$213.16,Genuine GM Parts 10341533 Rear Bumper Valance ...,Automotive
2,None,None,B001QTEKVO,JLM,(HID Xenon lights are designed to be at least ...,"(Will run for approx 2500 hours, Produces 2 to...",(https://images-na.ssl-images-amazon.com/image...,None,JLM HID Conversion Kit H13 (9008) Dual Tube B...,Automotive
3,None,"(B007KLMLRM, B007KLMNNE, B0085FOJ90, B0085FOAW...",B00HWI43Q0,Spec-D Tuning,(Brand new in original packaging. Exactly the ...,(Features 1 pair of Red & Smoked lens Tail Lig...,(https://images-na.ssl-images-amazon.com/image...,$8.63,Spec-D Tuning LT-E362RG-F2-APC New 3D Light Ba...,Automotive
4,None,None,B0050VHRFK,Oracle Lighting,(Oracle Flush LED switches combines styling an...,"(Heavy stainless steel construction, LED illum...",(https://images-na.ssl-images-amazon.com/image...,$19.95,Oracle Lighting LSGQ16FW White LED On/Off Flus...,Automotive


For some of the columns we have list values. We will replace them with Tuples, so then we could see if there are any duplicates.

In [16]:
common_pre_processing.convert_list_to_tuple().head()

,also_buy,also_view,asin,brand,description,feature,image,price,title,main_cat
0,None,None,B00ADZ3WUM,NSI,(Bumpersticker: A day without sunshine is like...,(Official Licensed Die-Cut Sticker Designed by...,None,$4.68,"NSI - A Day Without Sunshine is Like, Well, Ni...",Automotive
1,None,None,B005VII5IU,General Motors,(This is the official Genuine General Motors P...,(This is the official Genuine General Motors P...,None,$213.16,Genuine GM Parts 10341533 Rear Bumper Valance ...,Automotive
2,None,None,B001QTEKVO,JLM,(HID Xenon lights are designed to be at least ...,"(Will run for approx 2500 hours, Produces 2 to...",(https://images-na.ssl-images-amazon.com/image...,None,JLM HID Conversion Kit H13 (9008) Dual Tube B...,Automotive
3,None,"(B007KLMLRM, B007KLMNNE, B0085FOJ90, B0085FOAW...",B00HWI43Q0,Spec-D Tuning,(Brand new in original packaging. Exactly the ...,(Features 1 pair of Red & Smoked lens Tail Lig...,(https://images-na.ssl-images-amazon.com/image...,$8.63,Spec-D Tuning LT-E362RG-F2-APC New 3D Light Ba...,Automotive
4,None,None,B0050VHRFK,Oracle Lighting,(Oracle Flush LED switches combines styling an...,"(Heavy stainless steel construction, LED illum...",(https://images-na.ssl-images-amazon.com/image...,$19.95,Oracle Lighting LSGQ16FW White LED On/Off Flus...,Automotive


Finally, we will remove the duplicates rows.

In [17]:
df_common_pre_processed = common_pre_processing.remove_duplicated_rows()
df_common_pre_processed.head()

,also_buy,also_view,asin,brand,description,feature,image,price,title,main_cat
0,None,None,B00ADZ3WUM,NSI,(Bumpersticker: A day without sunshine is like...,(Official Licensed Die-Cut Sticker Designed by...,None,$4.68,"NSI - A Day Without Sunshine is Like, Well, Ni...",Automotive
1,None,None,B005VII5IU,General Motors,(This is the official Genuine General Motors P...,(This is the official Genuine General Motors P...,None,$213.16,Genuine GM Parts 10341533 Rear Bumper Valance ...,Automotive
2,None,None,B001QTEKVO,JLM,(HID Xenon lights are designed to be at least ...,"(Will run for approx 2500 hours, Produces 2 to...",(https://images-na.ssl-images-amazon.com/image...,None,JLM HID Conversion Kit H13 (9008) Dual Tube B...,Automotive
3,None,"(B007KLMLRM, B007KLMNNE, B0085FOJ90, B0085FOAW...",B00HWI43Q0,Spec-D Tuning,(Brand new in original packaging. Exactly the ...,(Features 1 pair of Red & Smoked lens Tail Lig...,(https://images-na.ssl-images-amazon.com/image...,$8.63,Spec-D Tuning LT-E362RG-F2-APC New 3D Light Ba...,Automotive
4,None,None,B0050VHRFK,Oracle Lighting,(Oracle Flush LED switches combines styling an...,"(Heavy stainless steel construction, LED illum...",(https://images-na.ssl-images-amazon.com/image...,$19.95,Oracle Lighting LSGQ16FW White LED On/Off Flus...,Automotive


In [18]:
df_common_pre_processed.shape[0] / df.shape[0]

0.9845252027167205

About 1.5% of the rows were duplicated.

#### Price Pre-Processing

We can check also that the price, is a string. And for it to be useful, should be converted into a float.

But you can find that some values in the price are not as expected. For example: 
```
    'a-box-inner{background-color:#fff}#alohaBuyBoxWidget .selected{background-color:#fffbf3;border-color:#e77600;box-shadow:0 0 3px rgba(228,121,17,.5)}#alohaBuyBoxWidget .contract-not-available{color:gray}#aloha-cart-popover .aloha-cart{height:auto;overflow:hidden}#aloha-cart-popover #aloha-cartInfo{float:left}#aloha-cart-popover #aloha-cart-details{float:right...
```

Which seems a part of CSS code.

So using the before function alone will encounter errors with those strings that are not representing a real price.

In [19]:
# Some examples of unexpected strings (about 35k of wrong values)
df_common_pre_processed[df_common_pre_processed.price.str.len() > 10].head(3)

,also_buy,also_view,asin,brand,description,feature,image,price,title,main_cat
173,None,None,B00SMX7WN0,WOW Trim,(An easy way to enhance your vehicles interior...,None,(https://images-na.ssl-images-amazon.com/image...,.a-box-inner{background-color:#fff}#alohaBuyBo...,Dash Overlay Kit ITEM# VWGF10A-SCF Volkswagen ...,Automotive
389,None,"(B000COUMBG, B000HLT0SK, B00029JKA0)",B000COW6C4,MSD,(The MSD universal replacement wire is 48 In. ...,None,None,.a-box-inner{background-color:#fff}#alohaBuyBo...,MSD 34063 Black 8.5mm 4' Long Spark Plug Wire,Automotive
434,None,None,B000C06YQM,Champ Labs,"(AIR FILTER,)",None,None,.a-box-inner{background-color:#fff}#alohaBuyBo...,Champ Labs AF7946 Air Filter,Automotive


In [20]:
price_pre_processing = PricePreProcessing(df_common_pre_processed)

Basically, we will try to apply converting the price into a float (by removing the dollar sign and then converting the data type).

In [21]:
df_pre_processed = price_pre_processing.pre_process()

In [22]:
# Now this value is as None
df_common_pre_processed.loc[173, :]

also_buy                                                    None
also_view                                                   None
asin                                                  B00SMX7WN0
brand                                                   WOW Trim
description    (An easy way to enhance your vehicles interior...
feature                                                     None
image          (https://images-na.ssl-images-amazon.com/image...
price                                                        NaN
title          Dash Overlay Kit ITEM# VWGF10A-SCF Volkswagen ...
main_cat                                              Automotive
Name: 173, dtype: object

In [23]:
df_common_pre_processed.isnull().sum(axis=0) / len(df)

also_buy       0.716662
also_view      0.650547
asin           0.000000
brand          0.029507
description    0.000000
feature        0.287128
image          0.481971
price          0.469452
title          0.000020
main_cat       0.000000
dtype: float64

## Pre-Processed Data

In [25]:
# We can basically re-write the before code with the following:
df = CommonPreProcessing(df).pre_process()
df = PricePreProcessing(df).pre_process()

In [26]:
df_into_csv_gzip(df, '../shared/data/amz_products_small_pre_processed')